In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import scipy
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

**Tehtävä 6**

Tarkastellaan luentomonisteen esimerkin 13. lineaarista regressiomallia

$ mpist_i = \beta_0 \, + \, \beta_1 \, matem_i \, + \, \beta_2 \, sukup_i \, + \, \beta_3 \, koulusij_i \, \\ + \, \beta_4 \, koulualue\_ita_i \, + \,       \beta_5 \, koulualue\_lansi_i \, + \, \beta_6 \, koulualue\_pohj_i \, + \,     \epsilon_i $

Lue aineisto Pythoniin ja laske käyttäen hyväksi luentomonisteen analyyttisiä tuloksia

- Parametrivektorin β suurimman uskottavuuden estimaatti
-  Mallin keskivirhe $ \mathbf{\hat{\sigma}} $
- Suurimman uskottavuuden estimaattien $ \mathbf{\hat{\beta}} $ keskivirheet
- Mallin selitysaste $ \mathbf{R^2} $

In [3]:
pisa = pd.read_csv("C:/Users/testi/Desktop/excel_files/pisa_demo02.csv", 
                   index_col=0)

In [4]:
pisa.head(3)

,mpist,sukup,SES,koulusij,koulualue,matem,aidink
107,569.254,poika,0.1322,kaupunki,Etela-Suomi,9,7
1261,631.804,poika,1.1144,kaupunki,Etela-Suomi,8,8
381,521.582,tytto,0.4400,kaupunki,Etela-Suomi,7,9


Luodaan design-matriisi X demolapussa olevan vihjeen avulla.

In [5]:
# Luodaan deisgn-matriisi.

mat = pisa["matem"].values

sp = pisa["sukup"].replace({"poika": 0, "tytto": 1}).values

sij = pisa["koulusij"].replace({"kaupunki": 0, "maaseutu": 1}).values

ita = np.where(pisa["koulualue"] == "Ita-Suomi", 1, 0)

lansi = np.where(pisa["koulualue"] == "Lansi-Suomi", 1, 0)

pohjoinen = np.where(pisa["koulualue"] == "Pohjois-Suomi", 1, 0)

intercept = np.ones(200, dtype=int)

X = pd.DataFrame({"intercept": intercept,
                  "mat": mat,
                  "sp": sp,
                  "sij": sij,
                  "ita": ita,
                  "lansi": lansi,
                  "pohjoinen": pohjoinen})

X.index += 1

In [6]:
X.head()

,intercept,mat,sp,sij,ita,lansi,pohjoinen
1,1,9,0,0,0,0,0
2,1,8,0,0,0,0,0
3,1,7,1,0,0,0,0
4,1,8,1,0,0,0,0
5,1,6,1,0,0,1,0


**Parametrivektorin β suurimman uskottavuuden estimaatti**

In [7]:
X_arr = X.values

In [8]:
y = pisa["mpist"].values

In [9]:
X_T_X = np.matmul(X_arr.T, X_arr)

In [10]:
X_T_y = np.matmul(X_arr.T, y)

In [11]:
betahat = np.matmul(np.linalg.inv(X_T_X), X_T_y)

In [12]:
betahat

array([334.61292837,  29.96626779, -17.34266536,  15.10478975,
        -3.57897317, -31.29944545, -11.89215235])

In [13]:
MLEs = pd.DataFrame({"MLE": betahat},
                     index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

In [14]:
MLEs

,MLE
intercept,334.612928
mat,29.966268
sp,-17.342665
sij,15.104790
ita,-3.578973
lansi,-31.299445
pohjoinen,-11.892152


**Mallin keskivirhe** $ \mathbf{\hat{\sigma}} \, $ on

$$ \mathbf{\hat{\sigma}} = \sqrt{\frac{(y - X\hat{\beta})^T \, (y - X\hat{\beta})}{n - p - 1}}. $$

Mallin keskivirheen laskemiseksi tarvitaan jäännökset

$$ \large\mathbf{\hat{\epsilon}} = y - \hat{y} $$

missä $ \mathbf{\hat{y}} $ ovat mallin sovitteet, jotka saadaan kaavalla

$$ \large\mathbf{\hat{y}} = X\hat{\beta} $$

missä X on design-matriisi ja $ \mathbf{\hat{\beta}} $ on parametrivektori.

- **Huomaa**, että keskivirheen kaavassa osoittajassa on residuaalivektorin transpoosi kertaa residuaalivektori.
- p = parametrivektorin $ \hat{\beta} $ parametrien (kertoimien) lukumäärä. **Huomaa**, että tähän ei lasketa intercept-termiä mukaan!

In [15]:
resid = y - np.matmul(X, betahat)

In [16]:
n = len(X)

In [17]:
p = betahat.shape[0] - 1

In [18]:
sigmahat = np.sqrt((np.matmul(resid.T, resid)) / (n - p - 1))

In [40]:
print(f"Mallin keskivirhe on: {sigmahat}")

Mallin keskivirhe on: 63.400172849497096


**Suurimman uskottavuuden estimaattien $ \mathbf{\hat{\beta}} $ keskivirheet**

Parametrien $ \mathbf{\hat{\beta_j}} $ keskivirheet ovat

$$ \sqrt{\widehat{Var}(\hat{\beta_j}}) = \sqrt{[\widehat{Cov}(\hat{\beta})}]_{jj} := se(\hat{\beta_j}), $$

$$ missä \, \widehat{Cov}(\hat{\beta}) = \sigma^2(X^TX)^{-1}. $$

In [20]:
covbeta = sigmahat**2 * np.linalg.inv(np.matmul(X_arr.T, X_arr))

In [21]:
sdbeta = np.sqrt(np.diagonal(covbeta))

In [22]:
pd.DataFrame({"Standard error": np.round(sdbeta, 2)},
              index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

,Standard error
intercept,24.68
mat,3.27
sp,9.03
sij,11.81
ita,15.26
lansi,10.89
pohjoinen,14.52


**Mallin selitysaste** $ \mathbf{R^2} $

$$ SST = \sum_{i=1}^n (y_i - \bar{y})^2 $$

$$ SSR = \sum_{i=1}^n (\hat{y_i} - \bar{y})^2 $$

$$ SSE = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

$$ \hat{y} = X\hat{\beta} $$

- SST on vasteen kokonaisvaihtelu
- SSR on regressiomallin selittämä vaihtelu
- SSE on selittämätön vaihtelu (jäännösvaihtelu)

$$ R^2 = \frac{SSR}{SST} = 1- \frac{SSE}{SST} $$

In [23]:
SST = sum((y - np.mean(y))**2)

In [24]:
yhat = np.matmul(X_arr, betahat)

In [25]:
SSE = sum((y - yhat)**2)

In [26]:
Rsquared = 1 - (SSE / SST)

In [41]:
print(f"Mallin selitysaste on: {Rsquared}")

Mallin selitysaste on: 0.3330070900240759


**Tehtävä 7**

Testaa t-testin avulla edellisen tehtävän lineaarisen mallin tapauksessa sukupuoleen liittyvää regressiokerrointa, eli nollahypoteesiä $ H_0 : \beta_2 = 0. $

t-testisuure lineaarisen mallin yksittäiselle regressiokertoimelle saadaan kaavalla

$$ t = \frac{\hat{\beta_j} - \beta_j}{se(\hat{\beta_j})}. $$

Huomaa, että $ \mathbf{\beta_j} $ on nollahypoteesin mukainen regressiokerroin (nolla) eli osoittajaan jää ainoastaan testattava regressiokerroin. 

In [28]:
t = betahat[2] / sdbeta[2]

In [43]:
print(f"Testisuure on: {t}")

Testisuure on: -1.9206433364970659


In [30]:
p_value = 2 * (1 - scipy.stats.t.cdf(np.abs(t), n-p-1))

In [44]:
print(f"p-arvo on: {p_value}")

p-arvo on: 0.05625059985174641


**Tehtävä 8**

Testaa F-testin avulla seuraavia hypoteeseja.
- $ H_0 : \beta_1 = \, ... \, = \beta_6 = 0 $ (keskiarvomalli on riittävä selittämään
vasteen arvoissa esiintyvää vaihtelua).
- $ H_1 : $ ainakin yksi regressiokerroin $ \neq 0 $.
- Tämän voidaan ajatella testaavan yleisesti regressiomallin merkitsevyyttä ja se voidaan laskea kaavalla

$$ \large F = \frac{\frac{R^2}{p - 1}}{\frac{1 - R^2}{n - p}}, $$

missä p on regressiokertoimien lukumäärä vakiotermi mukaan lukien.

In [104]:
F1 = (Rsquared / (7 - 1)) / ((1 - Rsquared) / (n - 7))

In [105]:
F1

16.059733022990848

In [108]:
critical_value1 = scipy.stats.f.ppf(0.99, 6, n-7)

In [109]:
critical_value1

2.896631146555669

- Koska F-testisuureen arvo on suurempi kuin kriittinen arvo 1 %:n merkitsevyystasolla, nollahypoteesi voidaan hylätä ja todeta, että ainakin yksi regressiokerroin on nollasta poikkeava.

- $ H_0 : \beta_4 = \beta_5 = \beta_6 = 0 $

Osittainen F-testi (partial F-test) voidaan laskea kaavalla

$$ \large F = \frac{\frac{SSE_{reduced} - SSE_{full}}{k}}{\frac{SSE_{full}}{n - p - 1}}, $$

missä k on niiden syötemuuttujien määrä, jotka poistettiin mallista ja p on regressiokertoimien lukumäärä.

$$ SSE = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

$$ \hat{y} = X\hat{\beta} $$

In [36]:
X_koulualue = X[["ita", "lansi", "pohjoinen"]]

In [37]:
betahat_koulualue = MLEs[4:7]

In [88]:
yhat_koulualue = np.matmul(X_koulualue.values, betahat_koulualue.values.T.ravel())

In [90]:
yhat_koulualue.shape

(200,)

In [92]:
SSE_koulualue = sum((y - yhat_koulualue)**2)

In [129]:
F_partial = ((SSE_koulualue - SSE) / 3) / (SSE / (n - 7))

In [130]:
F_partial

5057.954804632745

In [110]:
critical_value2 = scipy.stats.f.ppf(0.99, 3, n-7)

In [112]:
critical_value2

3.8846877608929344

- Koska F-testisuureen arvo on suurempi kuin kriittinen arvo 1 %:n merkitsevyystasolla, nollahypoteesi voidaan hylätä.

- $ H_0 : \beta_4 = \beta_6 $
- $ H_1 : \beta_4 \neq \beta_6 $

In [113]:
X_ita_pohjoinen = X[["ita", "pohjoinen"]]

In [117]:
betahat_ita_pohjoinen = MLEs.loc[["ita", "pohjoinen"]]

In [124]:
yhat_ita_pohjoinen = np.matmul(X_ita_pohjoinen.values, 
                               betahat_ita_pohjoinen.values.T.ravel())

In [126]:
yhat_ita_pohjoinen.shape

(200,)

In [127]:
SSE_ita_pohjoinen = sum((y - yhat_ita_pohjoinen)**2)

In [131]:
F_partial2 = ((SSE_ita_pohjoinen - SSE) / 2) / (SSE / (n - 7))

In [134]:
F_partial2

7370.66564927661

In [132]:
critical_value3 = scipy.stats.f.ppf(0.99, 2, n-7)

In [133]:
critical_value3

4.716823098028525

- Koska F-testisuureen arvo on suurempi kuin kriittinen arvo 1 %:n merkitsevyystasolla, nollahypoteesi voidaan hylätä.

**Todennäköisesti en tehnyt tehtävän 8 kohdissa b ja c sitä, mitä pyydettiin.**
- Ymmärtääkseni osittainen F-testi (partial F-test) vertaa pienempiä malleja (nested models) alkuperäiseen malliin, jossa on kaikki regressiokertoimet mukana.
- Jos $ H_0 $ hylätään (testisuure > kriittinen arvo), niin voidaan todeta, että kyseessä oleva nested model ei ole riittävä / parempi verrattuna alkuperäiseen malliin, jossa on kaikki regressiokertoimet mukana.
- Tämä ei mielestäni ole  verrattavissa demolapussa olevien sulkeiden sisältöön  tehtävien b ja c kohdalla.